<a href="https://colab.research.google.com/github/toferkey/Python_tutorial/blob/master/docs/examples/sam2_point_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting remote sensing imagery with point prompts

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_point_prompts.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_point_prompts.ipynb)

This notebook shows how to generate object masks from point prompts with the Segment Anything Model 2 (SAM 2).

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [1]:
 %pip install -U segment-geospatial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.4/488.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 9.2 M

## Import libraries

In [1]:
import leafmap
from samgeo import SamGeo2, regularize

## Create an interactive map

In [2]:
m = leafmap.Map(center=[52.9967591, -1.9620574], zoom=12, height="800px")
m.add_basemap("Satellite")
m

Map(center=[52.9967591, -1.9620574], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map. If no geometry is drawn, the default bounding box will be used.

In [3]:
if m.user_roi is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-1.974782, 53.004751, -1.930583, 52.974383]

In [4]:
image = r'C:\Users\RDGRLKDL\Documents\GAP_TGIC_Ideas\FY24\SAR_MS\S2_capella\UK_processing\S2_Stack_1125_B8B4B3_UK.tif'
leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox, zoom=18, source="Satellite", overwrite=True
)

Downloaded image 1/1221
Downloaded image 2/1221
Downloaded image 3/1221
Downloaded image 4/1221
Downloaded image 5/1221
Downloaded image 6/1221
Downloaded image 7/1221
Downloaded image 8/1221
Downloaded image 9/1221
Downloaded image 10/1221
Downloaded image 11/1221
Downloaded image 12/1221
Downloaded image 13/1221
Downloaded image 14/1221
Downloaded image 15/1221
Downloaded image 16/1221
Downloaded image 17/1221
Downloaded image 18/1221
Downloaded image 19/1221
Downloaded image 20/1221
Downloaded image 21/1221
Downloaded image 22/1221
Downloaded image 23/1221
Downloaded image 24/1221
Downloaded image 25/1221
Downloaded image 26/1221
Downloaded image 27/1221
Downloaded image 28/1221
Downloaded image 29/1221
Downloaded image 30/1221
Downloaded image 31/1221
Downloaded image 32/1221
Downloaded image 33/1221
Downloaded image 34/1221
Downloaded image 35/1221
Downloaded image 36/1221
Downloaded image 37/1221
Downloaded image 38/1221
Downloaded image 39/1221
Downloaded image 40/1221
Downloade

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [5]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=683582.0, center=[52.989567, -1.9526824999999999], controls=(ZoomControl(options=['position', 'zoom…

## Initialize SAM class

Set `automatic=False` to enable the `SAM2ImagePredictor`.

In [ ]:
sam = SamGeo2(
    model_id="sam2-hiera-large",
    automatic=False,
)

Specify the image to segment.

In [ ]:
sam.set_image(image)

## Segment the image

Use the `predict_by_points()` method to segment the image with specified point coordinates. You can use the draw tools to add place markers on the map. If no point is added, the default sample points will be used.


In [ ]:
if m.user_rois is not None:
    point_coords_batch = m.user_rois
else:
    point_coords_batch = [
        [-117.599896, 47.655345],
        [-117.59992, 47.655167],
        [-117.599928, 47.654974],
        [-117.599518, 47.655337],
    ]

Segment the objects using the point prompts and save the output masks.

In [ ]:
sam.predict_by_points(
    point_coords_batch=point_coords_batch,
    point_crs="EPSG:4326",
    output="mask.tif",
    dtype="uint8",
)

## Display the result

Add the segmented image to the map.

In [ ]:
m.add_raster("mask.tif", cmap="viridis", nodata=0, opacity=0.7, layer_name="Mask")
m

![image](https://github.com/user-attachments/assets/49e413b9-e159-4d72-bf23-a0318bc82d44)

## Use an existing vector dataset as points prompts

Alternatively, you can specify a file path or HTTP URL to a vector dataset containing point geometries.

In [ ]:
geojson = "https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson"

Display the vector dataawr on the map.

In [ ]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
m.add_circle_markers_from_xy(
    geojson, radius=3, color="red", fill_color="yellow", fill_opacity=0.8
)
m

![image](https://github.com/user-attachments/assets/f0d3ff1e-15fa-4bd3-ac15-637e8d63527d)

## Segment image with a vector dataset

Segment the image using the specified file path to the vector dataset.

In [ ]:
output_masks = "building_masks.tif"

In [ ]:
sam.predict_by_points(
    point_coords_batch=geojson,
    point_crs="EPSG:4326",
    output=output_masks,
    dtype="uint8",
    multimask_output=False,
)

Display the segmented masks on the map.

In [ ]:
m.add_raster(
    output_masks, cmap="jet", nodata=0, opacity=0.7, layer_name="Building masks"
)
m

![image](https://github.com/user-attachments/assets/262e1a31-1648-47d2-9e71-c85ab15b1a5c)

## Clean up the result

Remove small objects from the segmented masks, fill holes, and compute geometric properties.

In [ ]:
out_vector = "building_vector.geojson"
out_image = "buildings.tif"

In [ ]:
array, gdf = sam.region_groups(
    output_masks, min_size=200, out_vector=out_vector, out_image=out_image
)

In [ ]:
gdf.head()

![image](https://github.com/user-attachments/assets/af9ffa11-8ebe-4b42-8cba-3f5bcc4912f4)

## Regularize building footprints

Regularize the building footprints using the `regularize()` method.

In [ ]:
output_regularized = "building_regularized.geojson"
regularize(out_vector, output_regularized)

Display the regularized building footprints on the map.

In [ ]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
style = {
    "color": "#ffff00",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0,
}
m.add_raster(out_image, cmap="tab20", opacity=0.7, nodata=0, layer_name="Buildings")
m.add_vector(
    output_regularized, style=style, layer_name="Building regularized", info_mode=None
)
m

![image](https://github.com/user-attachments/assets/b39ee029-2089-45b8-8ac0-ba0d750cec22)

## Interactive segmentation

In [ ]:
sam.show_map()

![](https://github.com/user-attachments/assets/4f487505-6e89-4892-9a70-95ab0aa69cb6)